<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/create_ontol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install owlready2

     |████████████████████████████████| 20.8MB 1.2MB/s 
  Created wheel for owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20263007 sha256=f821d4afa80e9038d30fd56186bf432494d89075768c2eb5845e5d0b5a2a6592
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
Successfully built owlready2


In [2]:
!pip install rdflib

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 


In [3]:
from owlready2 import *
import gzip
import json


In [12]:
#default_world.set_backend(filename="file_back4.sqlite3", exclusive=False)

onto = get_ontology("http://www.semanticweb.org//Amazon_Ontol#")
with onto:
  class Article(Thing):
    pass

  class User(Thing):
    pass

  class Category(Thing):
    pass
  
  class Review(Thing):
    pass

  class Brand(Thing):
    pass

  class Main_Category(Category):
    pass

  class Other_Category(Category):
    pass

  AllDisjoint([Main_Category,Other_Category])

  class asin(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class title(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class feature(Article >> str):
    pass
  
  class description(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class price(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [float]

  class fromCategory(Article >> Category):
    pass

  class fromBrand(Article >> Brand):
    pass

  class alsoViewed(Article >> Article):
    pass

  class alsoBought(Article >> Article):
    pass

  class reviewerID(DataProperty, FunctionalProperty):
    domain = [User]
    range = [str]

  class vote(DataProperty,FunctionalProperty):
    domain = [Review]
    range = [int]

  class reviewText(DataProperty,FunctionalProperty):
    domain = [Review]
    range = [str]



  
  

In [13]:
onto.save("sortie.owl", format="ntriples")